In [1]:
import os
# Memory allocation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.5,max_split_size_mb:32"

In [2]:
import subprocess
import pickle
from PIL import Image
import struct

def send_data(data, output):
    """
    Send data to PIPE identified by output, only works reliably with bytes.
    """
    serialized_data = pickle.dumps(data, protocol=2)
    size = len(serialized_data)
    output.write(struct.pack('!I', size))
    output.write(serialized_data)
    output.flush()

def receive_data(input):
    """
    Receive data to PIPE identified by input, only works reliably with bytes.
    """
    size_data = input.read(4)
    if not size_data:  # No more data, return None
        return None
    size = struct.unpack('!I', size_data)[0]
    data = input.read(size)
    return pickle.loads(data)

In [4]:
import io
import pandas
import time

# Getting images from testing folder
def get_next_image():
    get_next_image.i = get_next_image.i + 1
    print(get_next_image.i)
    if get_next_image.i > 8:
        get_next_image.i = 1
    return Image.open('testing/' + str(get_next_image.i) + '.jpg')
get_next_image.i = 0

# Create subprocess
# Can be killed by sending None
process = subprocess.Popen(["envs/py3torch/bin/python", "faux_subprocess.py"],
                            stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
# Send parameters to the subprocess
send_data(bytes("../../../include/yolov5"), process.stdin)
send_data(bytes("../trained_models/yolov5-aug1st/best.pt"), process.stdin)
min_confidence = 0.4

while True:
    try:
        image = get_next_image()

        # Check if no more images
        if image is None:
            # Inform the subprocess to close
            send_data(None, process.stdin)
            break
        
        img_bytes = io.BytesIO()
        image.save(img_bytes, format='BMP')
        send_data(img_bytes.getvalue(), process.stdin)
        result = receive_data(process.stdout)
        result_df = pandas.read_json(result)

        # Check if problem with detection
        if result_df is None:
            print("Received blank detection, subprocess probably died")
            # Inform the subprocess to close
            send_data(None, process.stdin)
            break

        # Do something with the result DataFrame
        result_df = result_df.loc[result_df['confidence'] >= min_confidence]
        print(result_df)
        # for index, row in result_df.iterrows():
        #     print(row['name'], row['confidence'], row['xmax'], row['xmin'], row['ymax'], row['ymin'])

    except IOError as e:
        print("faux_subprocess pipe error: " + str(e))
        break
    except KeyboardInterrupt:
        # Inform the subprocess to close
        send_data(None, process.stdin)
        break
    except Exception as e:
        print("Exception: " + str(e))
        # Inform the subprocess to close
        send_data(None, process.stdin)
        break

# Clean up
process.stdin.close()
process.stdout.close()
process.stderr.close()
# process.terminate()
process.wait()

1
   class  confidence              name        xmax        xmin        ymax  \
0      8    0.665405       image_badge  227.463806  209.805847  193.547363   
1     10    0.662936    image_tommygun  248.547409  227.805817  196.159271   
2      6    0.656026        gman_image  122.804733  103.776062  131.551971   
3      6    0.631917        gman_image  337.001740  321.697479  173.797974   
4      9    0.606415  image_bootlegger  355.665955  339.229340  172.155457   
5     13    0.460242         qual_gate  186.114532   75.541374  244.703339   
6     10    0.404314    image_tommygun  203.416168  189.825882  164.036102   

         ymin  
0  145.699921  
1  145.157822  
2   94.202927  
3  143.160873  
4  141.198334  
5   66.521088  
6  138.287567  
2
   class  confidence              name        xmax        xmin        ymax  \
0      9    0.509201  image_bootlegger  349.805756  332.247467  163.102615   

         ymin  
0  130.894547  
3
   class  confidence            name        xmax    

0